lets implement a simple feed forward network that learns xor

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

First lets manually compute everything

In [3]:
# create dataset
# design matrix (input)
X = torch.Tensor([[0,0],[0,1], [1,0], [1,1]])
y = torch.tensor([[0], [1], [1], [0]])

W = torch.Tensor([[1,1],[1,1]])   # layer 1 weight matrix
c = torch.Tensor([0,-1])          # bias vector
w_1 = torch.Tensor([1,-2])        # weight vector

# params
# bias
b = 0

# print(W)
# print(c)
# print(w_1)
# print(X)

# first layer transformation
# first step: multiply input matrix by the first layers weight matrix
temp = torch.matmul(X,W)
print(temp)
# print(X @ W)              # note: @ same as matmul

# now, add bias vector c
temp += c
print(temp)

# note: all values in this space right now lie on a line with slope of 1
# as we move along the line the output needs to start at 0, rise to 1, and then fall to 0
#   a linear model cannot implement this! ( we will apply RELU to compute h)
m = nn.ReLU()
temp = m(temp)
print(temp)

# seccond layer transformation 
# output: multiply the transformation by the weight vector
out = torch.matmul(temp,w_1)
print("XOR output", out)

# assertion
# Compare with expected XOR output
expected_output = torch.Tensor([0, 1, 1, 0])
print("\nExpected XOR output:")
print(expected_output)

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [2., 2.]])
tensor([[ 0., -1.],
        [ 1.,  0.],
        [ 1.,  0.],
        [ 2.,  1.]])
tensor([[0., 0.],
        [1., 0.],
        [1., 0.],
        [2., 1.]])
XOR output tensor([0., 1., 1., 0.])

Expected XOR output:
tensor([0., 1., 1., 0.])


model time

In [28]:
# here we have a simple feedforward neural network
class XORNet(nn.Module):
    def __init__(self,**kwargs):
        super(XORNet, self).__init__(**kwargs)
        self.layer1 = nn.Linear(2, 2, dtype=torch.float32)
        self.layer2 = nn.Linear(2,1, dtype=torch.float32)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid() # output is binary
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        print("final dtype after sigmoid: ", x.dtype)

        return x

# Create the model
model = XORNet()
print(model)

XORNet(
  (layer1): Linear(in_features=2, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [31]:
# Training loop
for epoch in range(1000):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}')

final dtype after sigmoid:  torch.float32


RuntimeError: Expected floating point type for target with class probabilities, got Long